<a href="https://colab.research.google.com/github/Rahul-Singh-10/Google-Analytics-Ecommerce-Predition_Model/blob/main/03_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Imports**:
1.1 Import needed libaries and parquet file 

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 57.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5d8e0b94cb44079f72ae47f664d89929347654c8688e3db917c0080fb52f53bc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import random
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, StructType
from pyspark.sql.functions import split
from pyspark.sql.functions import *
from pyspark.sql.functions import window
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import missingno as msno
drive.mount('/content/drive')

In [ ]:
spark = SparkSession.builder.appName('Project').getOrCreate()
spark

In [ ]:
df = spark.read.parquet('/content/drive/MyDrive/df.parquet')

In [ ]:
df.show(n=5)

In [ ]:
df.select('fullVisitorId').distinct().count()

In [ ]:
df.withColumn('null_indicator',F.when(F.col('fullVisitorId').isNull(),1).otherwise(0)).groupBy('null_indicator').agg(F.count('*')).show()

In [ ]:
customers = df.sort('fullVisitorId')

## DATA TYPE CONVERSION/ FILLING MISSING DATA

In [ ]:
from pyspark.sql.types import StringType, BooleanType, IntegerType, DateType,FloatType 

In [ ]:
from pyspark.sql.functions import date_format
customers = customers.select("*",F.to_date('date','yyyyMMdd').alias('Date_fmt'))

In [ ]:
customers = customers.withColumn('bounces',customers.bounces.cast(IntegerType()))\
          .withColumn('hits',customers.hits.cast(IntegerType()))\
          .withColumn('pageviews',customers.pageviews.cast(IntegerType()))\
          .withColumn('visitNumber',customers.visitNumber.cast(IntegerType()))\
          .withColumn('action_type',customers.action_type.cast(IntegerType()))\
          .withColumn('promoIsClick',customers.promoIsClick.cast(BooleanType()))\
          .withColumn('promoIsView',customers.promoIsView.cast(BooleanType()))\
          .withColumn('hasSocialSourceReferral',customers.hasSocialSourceReferral.cast(BooleanType()))\
          .withColumn('socialNetwork',customers.socialNetwork.cast(BooleanType()))\
          .withColumn('timeOnSite',customers.timeOnSite.cast(FloatType()))\
          .withColumn('domContentLoadedTime',customers.domContentLoadedTime.cast(FloatType()))\
          .withColumn('domInteractiveTime',customers.domInteractiveTime.cast(FloatType()))\
          .withColumn('domainLookupTime',customers.domainLookupTime.cast(FloatType()))\
          .withColumn('redirectionTime',customers.redirectionTime.cast(FloatType()))\
          .withColumn('serverResponseTime',customers.serverResponseTime.cast(FloatType()))\
          .withColumn('isClick',customers.isClick.cast('int'))\
          .withColumn('totalTransactionRevenue',customers.totalTransactionRevenue.cast('int'))\
          .withColumn('transactions',customers.transactions.cast('int'))\


In [ ]:
customers.select('fullVisitorId').distinct().count()

714167

In [ ]:
customers.select('*').show()

In [ ]:
nafiller = ['bounces','pageviews','timeOnSite','totalTransactionRevenue','transactions','domContentLoadedTime','domInteractiveTime','domainLookuptime','redirectionTime','serverResponseTime','isClick']

customers = customers.fillna(0,subset=nafiller)

In [ ]:
customers = customers.drop('date','operatingSystem','medium','source','localProductPrice', 'localProductRevenue', 'productBrand','productListName', 'productListPosition','productSKU', 'exitScreenName','landingScreenName', 'contentGroupUniqueViews1','contentGroupUniqueViews2', 'contentGroupUniqueViews3', 'hasSocialSourceReferral', 'socialNetwork','referralPath','adContent','productPrice','productQuantity','productRevenue','eventLabel','eventCategory','isTrueDirect','isImpression')
customers.printSchema()

In [ ]:
customers = customers.withColumn('isClick',regexp_replace('isClick','true','1'))

customers.select('isClick').distinct().show()

In [ ]:
customers = customers.fillna('Unknown',subset='eventAction')

In [ ]:
customers = customers.withColumn('domain_total',F.col('domContentLoadedTime') + F.col('domInteractiveTime') + F.col('domainLookupTime') + F.col('redirectionTime'))

## Feature Engineering: Aggregation  
Create window functions to overlook and look back past 30 days to create a churn field indicated by 1 or 0. This will show whether user churned away from the platform or is a returning customer.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import days

In [ ]:
wp30d = Window.partitionBy('fullVisitorId').orderBy(F.unix_timestamp(F.col('Date_fmt'))/864000).rangeBetween(-30,-1)
wp15d = Window.partitionBy('fullVisitorId').orderBy(F.unix_timestamp(F.col('Date_fmt'))/864000).rangeBetween(-15,-1)
wp7d = Window.partitionBy('fullVisitorId').orderBy(F.unix_timestamp(F.col('Date_fmt'))/864000).rangeBetween(-7,-1)
wp3d = Window.partitionBy('fullVisitorId').orderBy(F.unix_timestamp(F.col('Date_fmt'))/864000).rangeBetween(-3,-1)
wpF30d = Window.partitionBy('fullVisitorId').orderBy(F.unix_timestamp(F.col('Date_fmt'))/864000).rangeBetween(1,30)
wpF15d = Window.partitionBy('fullVisitorId').orderBy(F.unix_timestamp(F.col('Date_fmt'))/864000).rangeBetween(1,15)

In [ ]:
customers = customers.withColumn('isClick',customers.isClick.cast('int'))

In [ ]:
customers = customers.withColumn('pageviews_L30d',F.sum('pageviews').over(wp30d))\
                      .withColumn('pageviews_L15d',F.sum('pageviews').over(wp15d))\
                      .withColumn('pageviews_L7d',F.sum('pageviews').over(wp7d))\
                      .withColumn('pageviews_L3d',F.sum('pageviews').over(wp3d))\
                      .withColumn('pageviews_D30d15d',F.col('pageviews_L30d') - F.col('pageviews_L15d'))\
                      .withColumn('pageviews_R30d15d',F.col('pageviews_L30d') / F.col('pageviews_L15d'))\
                      .withColumn('clicks_L30d',F.sum('isClick').over(wp30d))\
                      .withColumn('clicks_L15d',F.sum('isClick').over(wp15d))\
                      .withColumn('clicks_D30d15d',F.col('clicks_L30d') - F.col('clicks_L15d'))\
                      .withColumn('clicks_R30d15d',F.col('clicks_L30d') / F.col('clicks_L15d'))\
                      .withColumn('avgsession_L30d',F.mean('timeOnSite').over(wp30d))\
                      .withColumn('avgsession_L15d',F.mean('timeOnSite').over(wp15d))\
                      .withColumn('avgsession_D30d15d',F.col('avgsession_L30d') - F.col('avgsession_L15d'))\
                      .withColumn('avgsession_R30d15d',F.col('avgsession_L30d') / F.col('avgsession_L15d'))\
                      .withColumn('avgbrowstime_L30d',F.sum('pageviews_L30d').over(wp30d) / F.col('avgsession_L30d'))\
                      .withColumn('avgbrowstime_L15d',F.sum('pageviews_L15d').over(wp30d) / F.col('avgsession_L15d'))\
                      .withColumn('avgbrowstime_D30d15d', F.col('avgbrowstime_L30d') - F.col('avgbrowstime_L15d'))\
                      .withColumn('avgbrowstime_R30d15d',F.col('avgbrowstime_L30d') / F.col('avgbrowstime_L15d'))\
                      .withColumn('avgserverdelay_L30d',F.sum('serverResponseTime').over(wp30d) / F.col('avgsession_L30d'))\
                      .withColumn('avgserverdelay_L15d',F.sum('serverResponseTime').over(wp15d) / F.col('avgsession_L15d'))\
                      .withColumn('avgserverdelay_D30d15d',F.col('avgserverdelay_L30d') - F.col('avgserverdelay_L15d'))\
                      .withColumn('avgserverdelay_R30d15d',F.col('avgserverdelay_L30d') / F.col('avgserverdelay_L15d'))\
                      .withColumn('avgdomaintime_L30d',F.sum('domain_total').over(wp30d) / F.col('avgsession_L30d'))\
                      .withColumn('avgdomaintime_L15d',F.sum('domain_total').over(wp15d) / F.col('avgsession_L15d'))\
                      .withColumn('avgdomaintime_D30d15d',F.col('avgdomaintime_L30d') - F.col('avgdomaintime_L15d'))\
                      .withColumn('avgdomaintime_R30d15d',F.col('avgdomaintime_L30d') / F.col('avgdomaintime_L15d'))\
                      .withColumn('avg$_L30d',F.sum('totalTransactionRevenue').over(wp30d) / F.sum('transactions').over(wp30d))\
                      .withColumn('avg$_L15d',F.sum('totalTransactionRevenue').over(wp15d) / F.sum('transactions').over(wp15d))\
                      .withColumn('avg$_D30d15d',F.col('avg$_L30d') - F.col('avg$_L15d'))\
                      .withColumn('avg$_R30d15d',F.col('avg$_L30d') / F.col('avg$_L15d'))\
                      .withColumn('Maxsession_durationL30d',F.max('timeOnSite').over(wp30d))\
                      .withColumn('Maxsession_durationL15d',F.max('timeOnSite').over(wp15d))\
        

# Hot Encoding
Encode Categorical Variables to Numerical columns as the model will only work on numerical values not booleans.


In [ ]:
dis_cg = customers.select('channelGrouping').distinct().rdd.flatMap(lambda x:x).collect()
dis_ea = customers.select('eventAction').distinct().rdd.flatMap(lambda x:x).collect()
dis_dc = customers.select('deviceCategory').distinct().rdd.flatMap(lambda x:x).collect()

In [ ]:
print(dis_dc)
print(dis_cg)
print(dis_ea)

['desktop', 'mobile', 'tablet']
['Display', 'Affiliates', 'Social', 'Direct', 'Referral', '(Other)', 'Paid Search', 'Organic Search']
['Product Click', 'Add to Cart', 'Unknown', 'Remove from Cart', 'Quickview Click', 'Onsite Click', 'Promotion Click']


In [ ]:
from pyspark.sql.functions import udf, col

In [ ]:
for uniqval in dis_cg:
  func = udf(lambda item: 1 if item == uniqval else 0, IntegerType())
  new_col = 'channelGrouping' + '_' + uniqval
  customers = customers.withColumn(new_col,func(customers.channelGrouping))


for uniqval1 in dis_ea:
  func1 = udf(lambda item: 1 if item == uniqval1 else 0, IntegerType())
  new_col1 = 'eventAction' + '_' + uniqval1
  customers = customers.withColumn(new_col1,func1(col('eventAction')))  

for uniqval2 in dis_dc:
  func2 = udf(lambda item: 1 if item == uniqval2 else 0, IntegerType())
  new_col2 = 'deviceCategory' + '_' + uniqval2
  customers = customers.withColumn(new_col2,func2(col('deviceCategory')))  


#customers.show()

In [ ]:
customers = customers.withColumn('channelGrouping_Display_L30d',F.sum('channelGrouping_Display').over(wp30d))\
                      .withColumn('channelGrouping_Display_L15d',F.sum('channelGrouping_Display').over(wp15d))\
                      .withColumn('channelGrouping_Affiliates_L30d',F.sum('channelGrouping_Affiliates').over(wp30d))\
                      .withColumn('channelGrouping_Affiliates_L15d',F.sum('channelGrouping_Affiliates').over(wp15d))\
                      .withColumn('channelGrouping_Social_L30d',F.sum('channelGrouping_Social').over(wp30d))\
                      .withColumn('channelGrouping_Social_L15d',F.sum('channelGrouping_Social').over(wp15d))\
                      .withColumn('channelGrouping_Direct_L30d',F.sum('channelGrouping_Direct').over(wp30d))\
                      .withColumn('channelGrouping_Direct_L15d',F.sum('channelGrouping_Direct').over(wp15d))\
                      .withColumn('channelGrouping_Referral_L30d',F.sum('channelGrouping_Referral').over(wp30d))\
                      .withColumn('channelGrouping_Referral_L15d',F.sum('channelGrouping_Referral').over(wp15d))\
                      .withColumn('channelGrouping_Other_L30d',F.sum('channelGrouping_(Other)').over(wp30d))\
                      .withColumn('channelGrouping_Other_L15d',F.sum('channelGrouping_(Other)').over(wp15d))\
                      .withColumn('channelGrouping_Paid_Search_L30d',F.sum('channelGrouping_Paid Search').over(wp30d))\
                      .withColumn('channelGrouping_Paid_Search_L15d',F.sum('channelGrouping_Paid Search').over(wp15d))\
                      .withColumn('channelGrouping_Organic_Search_L30d',F.sum('channelGrouping_Organic Search').over(wp30d))\
                      .withColumn('channelGrouping_Organic_Search_L15d',F.sum('channelGrouping_Organic Search').over(wp15d))\
                      .withColumn('eventAction_Product_Click_L30d',F.sum('eventAction_Product Click').over(wp30d))\
                      .withColumn('eventAction_Product_Click_L15d',F.sum('eventAction_Product Click').over(wp15d))\
                      .withColumn('eventAction_Add_to_Cart_L30d',F.sum('eventAction_Add to Cart').over(wp30d))\
                      .withColumn('eventAction_Add_to_Cart_L15d',F.sum('eventAction_Add to Cart').over(wp15d))\
                      .withColumn('eventAction_Unknown_L30d',F.sum('eventAction_Unknown').over(wp30d))\
                      .withColumn('eventAction_Unknown_L15d',F.sum('eventAction_Unknown').over(wp15d))\
                      .withColumn('eventAction_Remove_from_Cart_L30d',F.sum('eventAction_Remove from Cart').over(wp30d))\
                      .withColumn('eventAction_Remove_from_Cart_L15d',F.sum('eventAction_Remove from Cart').over(wp15d))\
                      .withColumn('eventAction_Quickview_Click_L30d',F.sum('eventAction_Quickview Click').over(wp30d))\
                      .withColumn('eventAction_Quickview_Click_L15d',F.sum('eventAction_Quickview Click').over(wp15d))\
                      .withColumn('eventAction_Onsite_Click_L30d',F.sum('eventAction_Onsite Click').over(wp30d))\
                      .withColumn('eventAction_Onsite_Click_L15d',F.sum('eventAction_Onsite Click').over(wp15d))\
                      .withColumn('eventAction_Promotion_Click_L30d',F.sum('eventAction_Promotion Click').over(wp30d))\
                      .withColumn('eventAction_Promotion_Click_L15d',F.sum('eventAction_Promotion Click').over(wp15d))\
                      .withColumn('deviceCategory_desktop_L30d',F.sum('deviceCategory_desktop').over(wp30d))\
                      .withColumn('deviceCategory_desktop_L15d',F.sum('deviceCategory_desktop').over(wp15d))\
                      .withColumn('deviceCategory_mobile_L30d',F.sum('deviceCategory_mobile').over(wp30d))\
                      .withColumn('deviceCategory_mobile_L15d',F.sum('deviceCategory_mobile').over(wp15d))\
                      .withColumn('deviceCategory_tablet_L30d',F.sum('deviceCategory_tablet').over(wp30d))\
                      .withColumn('deviceCategory_tablet_L15d',F.sum('deviceCategory_tablet').over(wp15d))\
                      .withColumn('visitFreq_L30d',F.max('visitNumber').over(wp30d))\
                      .withColumn('visitFreq_L15d',F.max('visitNumber').over(wp15d))\
                      .withColumn('visitFreq_N30d',F.max('visitNumber').over(wpF30d))\
                      .withColumn('visitFreq_N15d',F.max('visitNumber').over(wpF15d))\
                      .withColumn('Churn_$30d',(F.when(F.sum('totalTransactionRevenue').over(wpF30d)>0,0).otherwise(1)))\
                      .withColumn('Churn_$15d',(F.when(F.sum('totalTransactionRevenue').over(wpF15d)>0,0).otherwise(1)))\
                      .withColumn('prev_date',F.lag('Date_fmt').over(Window.partitionBy("fullVisitorID").orderBy('Date_fmt')))\
                      .withColumn('time_lastvisit',F.datediff(F.col('Date_fmt'),F.col('prev_date')))\
                      .withColumn('visit_capture',F.size(F.collect_set('visitID').over(wpF30d)))\
                      .withColumn('Churn_vc30d',F.when(F.col('visit_capture')>1,0).otherwise(1))\
                      .withColumn('Churn_vf30d',F.when(F.col('visitFreq_N30d')>1,0).otherwise(1))

In [ ]:
customers.groupBy('Churn_vc30d').agg(F.count('*')).show()#1 is churn and 0 is stay ; error in feature

In [ ]:
customers.groupBy('Churn_vf30d').agg(F.count('*')).show()

In [ ]:
customers.filter(F.col('fullVisitorId') == '0824839726118485274').select('fullVisitorId','Date_fmt','prev_date','time_lastvisit','visit_capture','Churn_vc30d').show()

In [ ]:
customers.groupby('time_lastvisit').agg(F.count('*').alias('CIP')).orderBy(F.col('CIP').desc()).show()

In [ ]:
customers.select('*').groupBy('fullVisitorID').agg(F.max('visitNumber')).show()

In [ ]:
customers = customers.drop('channelGrouping',
 'deviceCategory',
 'country',
 'newVisits',
 'pageviews',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactions',
 'visitStartTime',
 'isClick',
 'action_type',
 'eventAction',
 'isFatal',
 'domContentLoadedTime',
 'domInteractiveTime',
 'domainLookupTime',
 'redirectionTime',
 'serverResponseTime',
 'promoIsClick',
 'promoIsView',
 'channelGrouping_(Other)',
 'channelGrouping_Paid Search',
 'channelGrouping_Organic Search',
 'eventAction_Product Click',
 'eventAction_Add to Cart',
 'eventAction_Remove from Cart',
 'eventAction_Quickview Click',
 'eventAction_Onsite Click',
 'eventAction_Promotion Click')

## Final Dataset to parquet file for saving

In [ ]:
customers.write.parquet('/content/drive/MyDrive/finaldf1.parquet',mode='overwrite')

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/finaldf1.parquet')
pd.set_option("display.max_columns", None)

In [ ]:
df.dtypes

In [ ]:
df['Date_fmt'] = pd.to_datetime(df['Date_fmt'])

In [ ]:
df = df[df['Date_fmt'] > '2016-08-31'].set_index('Date_fmt').fillna(-9999)

In [ ]:
df.fullVisitorId.value_counts()

In [ ]:
df[df.fullVisitorId == '0069905060737267712']

In [ ]:
df[df.fullVisitorId == '0001059349366430257']

In [ ]:
df[df.fullVisitorId == '999583991723927576']